In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import polars as pl
from xgboost import XGBClassifier
import gc

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Print filepaths and save to file_dirs 
file_dirs = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_dirs.append(dirname + '/' + filename)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [9]:
train = pl.read_csv(file_dirs[1])
target = pl.read_csv(file_dirs[2])

In [10]:
train = train.drop("id")
target = target.drop("id")

In [12]:
category_columns = [col for col, dtype in zip(train.columns, train.dtypes) if dtype == pl.Utf8]

In [13]:
target = target.with_columns(pl.lit(0).cast(pl.Int64).alias('Response'))

In [14]:
df = pl.concat([train, target])

In [15]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
str,i64,i64,f64,i64,str,str,f64,f64,i64,i64
"""Male""",21,1,35.0,0,"""1-2 Year""","""Yes""",65101.0,124.0,187,0
"""Male""",43,1,28.0,0,"""> 2 Years""","""Yes""",58911.0,26.0,288,1
"""Female""",25,1,14.0,1,"""< 1 Year""","""No""",38043.0,152.0,254,0
"""Female""",35,1,1.0,0,"""1-2 Year""","""Yes""",2630.0,156.0,76,0
"""Female""",36,1,15.0,1,"""1-2 Year""","""No""",31951.0,152.0,294,0
…,…,…,…,…,…,…,…,…,…,…
"""Male""",57,1,28.0,0,"""1-2 Year""","""Yes""",51661.0,124.0,109,0
"""Male""",28,1,50.0,1,"""< 1 Year""","""No""",25651.0,152.0,184,0
"""Male""",47,1,33.0,1,"""1-2 Year""","""No""",2630.0,138.0,63,0


In [16]:
df = df.with_columns([
    pl.col('Region_Code').cast(int),
    pl.col('Annual_Premium').cast(int),
    pl.col('Policy_Sales_Channel').cast(int)
])

In [17]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
str,i64,i64,i64,i64,str,str,i64,i64,i64,i64
"""Male""",21,1,35,0,"""1-2 Year""","""Yes""",65101,124,187,0
"""Male""",43,1,28,0,"""> 2 Years""","""Yes""",58911,26,288,1
"""Female""",25,1,14,1,"""< 1 Year""","""No""",38043,152,254,0
"""Female""",35,1,1,0,"""1-2 Year""","""Yes""",2630,156,76,0
"""Female""",36,1,15,1,"""1-2 Year""","""No""",31951,152,294,0
…,…,…,…,…,…,…,…,…,…,…
"""Male""",57,1,28,0,"""1-2 Year""","""Yes""",51661,124,109,0
"""Male""",28,1,50,1,"""< 1 Year""","""No""",25651,152,184,0
"""Male""",47,1,33,1,"""1-2 Year""","""No""",2630,138,63,0


In [18]:
df = df.with_columns([
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Annual_Premium'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Annual_Premium'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Age'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Age'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vehicle_Damage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vehicle_Damage'),
    (pl.Series(pd.factorize((df['Previously_Insured'].cast(str) + df['Vintage'].cast(str)).to_numpy())[0])).alias('Previously_Insured_Vintage')
])

In [19]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
str,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64
"""Male""",21,1,35,0,"""1-2 Year""","""Yes""",65101,124,187,0,0,0,0,0
"""Male""",43,1,28,0,"""> 2 Years""","""Yes""",58911,26,288,1,1,1,0,1
"""Female""",25,1,14,1,"""< 1 Year""","""No""",38043,152,254,0,2,2,1,2
"""Female""",35,1,1,0,"""1-2 Year""","""Yes""",2630,156,76,0,3,0,0,3
"""Female""",36,1,15,1,"""1-2 Year""","""No""",31951,152,294,0,4,3,1,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Male""",57,1,28,0,"""1-2 Year""","""Yes""",51661,124,109,0,8476,0,0,518
"""Male""",28,1,50,1,"""< 1 Year""","""No""",25651,152,184,0,28152,2,1,463
"""Male""",47,1,33,1,"""1-2 Year""","""No""",2630,138,63,0,18,3,1,115


In [20]:
tar='Response'

In [21]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [22]:
train = df[:train.shape[0]].to_pandas()
test = df[train.shape[0]:].to_pandas()

In [23]:
train =reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage of dataframe is 1316.62 MB
Memory usage after optimization is: 263.32 MB
Decreased by 80.0%
Memory usage of dataframe is 877.75 MB
Memory usage after optimization is: 175.55 MB
Decreased by 80.0%


In [24]:
initial_features = test.columns

print(initial_features)

Index(['Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage', 'Response',
       'Previously_Insured_Annual_Premium', 'Previously_Insured_Vehicle_Age',
       'Previously_Insured_Vehicle_Damage', 'Previously_Insured_Vintage'],
      dtype='object')


In [25]:
df

Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
str,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64
"""Male""",21,1,35,0,"""1-2 Year""","""Yes""",65101,124,187,0,0,0,0,0
"""Male""",43,1,28,0,"""> 2 Years""","""Yes""",58911,26,288,1,1,1,0,1
"""Female""",25,1,14,1,"""< 1 Year""","""No""",38043,152,254,0,2,2,1,2
"""Female""",35,1,1,0,"""1-2 Year""","""Yes""",2630,156,76,0,3,0,0,3
"""Female""",36,1,15,1,"""1-2 Year""","""No""",31951,152,294,0,4,3,1,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Male""",57,1,28,0,"""1-2 Year""","""Yes""",51661,124,109,0,8476,0,0,518
"""Male""",28,1,50,1,"""< 1 Year""","""No""",25651,152,184,0,28152,2,1,463
"""Male""",47,1,33,1,"""1-2 Year""","""No""",2630,138,63,0,18,3,1,115


In [26]:
for i in initial_features:
    print(i, ':', len(df.unique(i)))

Gender : 2
Age : 66
Driving_License : 2
Region_Code : 53
Previously_Insured : 2
Vehicle_Age : 3
Vehicle_Damage : 2
Annual_Premium : 53958
Policy_Sales_Channel : 154
Vintage : 290
Response : 2
Previously_Insured_Annual_Premium : 99682
Previously_Insured_Vehicle_Age : 6
Previously_Insured_Vehicle_Damage : 4
Previously_Insured_Vintage : 580


In [27]:
categorical_features = [col for col in initial_features if pd.concat([train[col], test[col]]).nunique() < 10]
print(categorical_features)
categorical_features.remove('Response')

['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Response', 'Previously_Insured_Vehicle_Age', 'Previously_Insured_Vehicle_Damage']


In [28]:
numerical_features = list(set(initial_features) - set(categorical_features))

print(numerical_features)
numerical_features.remove('Response')

['Age', 'Vintage', 'Previously_Insured_Vintage', 'Response', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Previously_Insured_Annual_Premium']


In [29]:
trys = ['Policy_Sales_Channel','Vintage','Age','Previously_Insured_Vintage','Region_Code']

In [30]:
isinit = categorical_features.copy()

In [31]:
for i in range(1, len(trys)+1):
    isinit.append(trys[i-1])
    globals()[f'categorical_features{i}'] = isinit.copy()
    globals()[f'numerical_features{i}'] = list(set(initial_features) - set(globals()[f'categorical_features{i}']))
    globals()[f'numerical_features{i}'] .remove('Response')

In [32]:
print(categorical_features)
print(numerical_features)

['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Previously_Insured_Vehicle_Age', 'Previously_Insured_Vehicle_Damage']
['Age', 'Vintage', 'Previously_Insured_Vintage', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Previously_Insured_Annual_Premium']


In [33]:
for i in range(1,len(trys)+1):
    print(i)
    print(globals()[f'categorical_features{i}'])
    print(globals()[f'numerical_features{i}'])

1
['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Previously_Insured_Vehicle_Age', 'Previously_Insured_Vehicle_Damage', 'Policy_Sales_Channel']
['Age', 'Vintage', 'Previously_Insured_Vintage', 'Region_Code', 'Annual_Premium', 'Previously_Insured_Annual_Premium']
2
['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Previously_Insured_Vehicle_Age', 'Previously_Insured_Vehicle_Damage', 'Policy_Sales_Channel', 'Vintage']
['Age', 'Previously_Insured_Vintage', 'Region_Code', 'Annual_Premium', 'Previously_Insured_Annual_Premium']
3
['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Previously_Insured_Vehicle_Age', 'Previously_Insured_Vehicle_Damage', 'Policy_Sales_Channel', 'Vintage', 'Age']
['Previously_Insured_Vintage', 'Region_Code', 'Annual_Premium', 'Previously_Insured_Annual_Premium']
4
['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Previousl

In [34]:
ddf = train.copy()
x= ddf.drop(columns=['Response'],axis=1)
y= ddf['Response']

In [35]:
test = test.drop(columns=['Response'],axis=1)

In [36]:
x[categorical_features] = x[categorical_features].astype("category")
test[categorical_features] = test[categorical_features].astype("category")

In [37]:
X=x

In [38]:
X

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Previously_Insured_Annual_Premium,Previously_Insured_Vehicle_Age,Previously_Insured_Vehicle_Damage,Previously_Insured_Vintage
0,Male,21,1,35,0,1-2 Year,Yes,65101,124,187,0,0,0,0
1,Male,43,1,28,0,> 2 Years,Yes,58911,26,288,1,1,0,1
2,Female,25,1,14,1,< 1 Year,No,38043,152,254,2,2,1,2
3,Female,35,1,1,0,1-2 Year,Yes,2630,156,76,3,0,0,3
4,Female,36,1,15,1,1-2 Year,No,31951,152,294,4,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,Male,48,1,6,0,1-2 Year,Yes,27412,26,218,5210,0,0,144
11504794,Female,26,1,36,0,< 1 Year,Yes,29509,152,115,23274,4,0,176
11504795,Female,29,1,32,1,< 1 Year,No,2630,152,189,18,2,1,456
11504796,Female,51,1,28,0,1-2 Year,Yes,48443,26,274,14121,0,0,124


In [39]:
for i in range(1,len(trys)+1):
    print(i)
    globals()[f'X{i}'] = X[globals()[f'categorical_features{i}'].copy()].astype("category")
    globals()[f'test{i}'] = test[globals()[f'categorical_features{i}'].copy()].astype("category")

1
2
3
4
5


In [40]:
X['Previously_Insured_Vintage']

0             0
1             1
2             2
3             3
4             4
           ... 
11504793    144
11504794    176
11504795    456
11504796    124
11504797    456
Name: Previously_Insured_Vintage, Length: 11504798, dtype: int16

In [41]:
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'class_names': [0, 1],
    'learning_rate': 0.075,
    'iterations': 3000,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'max_leaves': 512,
    'fold_permutation_block': 64,
    'task_type': 'GPU',
    'random_seed': 42,
    'verbose': False,
    'allow_writing_files': False
}

In [42]:
import catboost as cb
from sklearn.model_selection import train_test_split, StratifiedKFold

In [43]:
skfold = StratifiedKFold(5, shuffle=True, random_state=42)

In [44]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, classification_report

In [45]:
count_ones = (y == 1).sum()
count_zeros = (y == 0).sum()
total_count = len(y)

# 1과 0의 비율을 계산
ratio_ones = count_ones / total_count
ratio_zeros = count_zeros / total_count

print(f"1의 비율: {ratio_ones:.2f}")
print(f"0의 비율: {ratio_zeros:.2f}")

1의 비율: 0.12
0의 비율: 0.88


In [46]:
len(X)

11504798

In [47]:
sample_ratio = 0.001

In [48]:
X_sample, _, y_sample, _ = train_test_split(X, y, test_size=(1 - sample_ratio), stratify=y, random_state=42)

In [49]:
for i in range(1,len(trys)+1):
    globals()[f'X_sample{i}'], _, globals()[f'y_sample{i}'], _ = train_test_split(globals()[f'X{i}'], y, test_size=(1 - sample_ratio), stratify=y, random_state=42)

In [50]:
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'class_names': [0, 1],
    'learning_rate': 0.075,
    'iterations': 700,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'max_leaves': 512,
    'fold_permutation_block': 64,
    'task_type': 'GPU',
    'random_seed': 42,
    'verbose': False,
    'allow_writing_files': False
}

In [52]:
X=X_sample
y=y_sample
oof_preds = []
oof_aucs = []

test_pool = cb.Pool(test.astype(str), cat_features=X.columns.values)

for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
    X_train_pool = cb.Pool(X_train.astype(str), y_train, cat_features=X.columns.values)
    X_test_pool = cb.Pool(X_test.astype(str), y_test, cat_features=X.columns.values)
    
    cat_clf = cb.CatBoostClassifier(**cat_params)
    cat_clf = cat_clf.fit(X=X_train_pool,
                          eval_set=X_test_pool,
                          verbose=10,
                          early_stopping_rounds=200)
    
    test_pred = cat_clf.predict_proba(test_pool)[:, 1]
    
    oof_preds.append(test_pred)
    auc = cat_clf.best_score_['validation']['AUC']
    oof_aucs.append(auc)
    print(f"\n---- Fold {fold}: ROC-AUC Score: {auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    del X_train_pool, X_test_pool
    del cat_clf
    gc.collect()

auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} \xB1 {auc_std:.6f}\n")

test_pred_cat = np.mean(oof_preds, axis=0)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8327298	best: 0.8327298 (0)	total: 33.5ms	remaining: 23.4s
10:	test: 0.8597700	best: 0.8597700 (10)	total: 1.79s	remaining: 1m 52s
20:	test: 0.8665491	best: 0.8665491 (20)	total: 4.04s	remaining: 2m 10s
30:	test: 0.8648069	best: 0.8665491 (20)	total: 6.28s	remaining: 2m 15s
40:	test: 0.8647482	best: 0.8665491 (20)	total: 8.53s	remaining: 2m 17s
50:	test: 0.8652017	best: 0.8665491 (20)	total: 10.8s	remaining: 2m 17s
60:	test: 0.8655659	best: 0.8665491 (20)	total: 13s	remaining: 2m 16s
70:	test: 0.8660378	best: 0.8665491 (20)	total: 15.3s	remaining: 2m 15s
80:	test: 0.8653164	best: 0.8665491 (20)	total: 17.6s	remaining: 2m 14s
90:	test: 0.8654241	best: 0.8665491 (20)	total: 19.8s	remaining: 2m 12s
100:	test: 0.8651054	best: 0.8665491 (20)	total: 22s	remaining: 2m 10s
110:	test: 0.8638359	best: 0.8665491 (20)	total: 24.2s	remaining: 2m 8s
120:	test: 0.8636993	best: 0.8665491 (20)	total: 26.5s	remaining: 2m 6s
130:	test: 0.8643297	best: 0.8665491 (20)	total: 28.7s	remaining: 2m 

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8310707	best: 0.8310707 (0)	total: 40ms	remaining: 28s
10:	test: 0.8554021	best: 0.8554021 (10)	total: 1.39s	remaining: 1m 27s
20:	test: 0.8595450	best: 0.8596921 (15)	total: 3.65s	remaining: 1m 58s
30:	test: 0.8613783	best: 0.8624036 (27)	total: 5.91s	remaining: 2m 7s
40:	test: 0.8627214	best: 0.8628877 (39)	total: 8.17s	remaining: 2m 11s
50:	test: 0.8635549	best: 0.8635549 (50)	total: 10.3s	remaining: 2m 11s
60:	test: 0.8642010	best: 0.8644514 (58)	total: 12.6s	remaining: 2m 11s
70:	test: 0.8656552	best: 0.8656552 (70)	total: 14.8s	remaining: 2m 11s
80:	test: 0.8661945	best: 0.8662015 (79)	total: 17.1s	remaining: 2m 10s
90:	test: 0.8654801	best: 0.8669685 (84)	total: 19.4s	remaining: 2m 9s
100:	test: 0.8649846	best: 0.8669685 (84)	total: 21.6s	remaining: 2m 8s
110:	test: 0.8651737	best: 0.8669685 (84)	total: 23.8s	remaining: 2m 6s
120:	test: 0.8650599	best: 0.8669685 (84)	total: 26.1s	remaining: 2m 4s
130:	test: 0.8653575	best: 0.8669685 (84)	total: 28.3s	remaining: 2m 2s


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8486300	best: 0.8486300 (0)	total: 38.6ms	remaining: 27s
10:	test: 0.8694164	best: 0.8694164 (10)	total: 1.83s	remaining: 1m 54s
20:	test: 0.8710606	best: 0.8716096 (12)	total: 4.11s	remaining: 2m 12s
30:	test: 0.8712909	best: 0.8717330 (28)	total: 6.36s	remaining: 2m 17s
40:	test: 0.8717321	best: 0.8718363 (33)	total: 8.62s	remaining: 2m 18s
50:	test: 0.8714966	best: 0.8720491 (43)	total: 10.9s	remaining: 2m 18s
60:	test: 0.8709652	best: 0.8720491 (43)	total: 13.2s	remaining: 2m 18s
70:	test: 0.8710370	best: 0.8720491 (43)	total: 15.5s	remaining: 2m 16s
80:	test: 0.8705344	best: 0.8720491 (43)	total: 17.7s	remaining: 2m 14s
90:	test: 0.8713644	best: 0.8720491 (43)	total: 19.9s	remaining: 2m 13s
100:	test: 0.8716586	best: 0.8720491 (43)	total: 22.2s	remaining: 2m 11s
110:	test: 0.8715693	best: 0.8720491 (43)	total: 24.3s	remaining: 2m 9s
120:	test: 0.8715098	best: 0.8720491 (43)	total: 26.6s	remaining: 2m 7s
130:	test: 0.8708566	best: 0.8720491 (43)	total: 28.4s	remaining: 2

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8340291	best: 0.8340291 (0)	total: 44.1ms	remaining: 30.8s
10:	test: 0.8501709	best: 0.8501709 (10)	total: 1.45s	remaining: 1m 30s
20:	test: 0.8615780	best: 0.8615780 (20)	total: 3.69s	remaining: 1m 59s
30:	test: 0.8620087	best: 0.8622906 (22)	total: 5.95s	remaining: 2m 8s
40:	test: 0.8620113	best: 0.8625060 (35)	total: 8.21s	remaining: 2m 11s
50:	test: 0.8618599	best: 0.8625060 (35)	total: 10.5s	remaining: 2m 13s
60:	test: 0.8614641	best: 0.8625060 (35)	total: 12.8s	remaining: 2m 13s
70:	test: 0.8611279	best: 0.8625060 (35)	total: 14.8s	remaining: 2m 11s
80:	test: 0.8612383	best: 0.8625060 (35)	total: 17.1s	remaining: 2m 10s
90:	test: 0.8613276	best: 0.8625060 (35)	total: 19.3s	remaining: 2m 9s
100:	test: 0.8615254	best: 0.8625060 (35)	total: 21.5s	remaining: 2m 7s
110:	test: 0.8614002	best: 0.8625060 (35)	total: 23.8s	remaining: 2m 6s
120:	test: 0.8611918	best: 0.8625060 (35)	total: 25.9s	remaining: 2m 3s
130:	test: 0.8614352	best: 0.8625060 (35)	total: 28.1s	remaining: 2m

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8166127	best: 0.8166127 (0)	total: 225ms	remaining: 2m 37s
10:	test: 0.8571261	best: 0.8571261 (10)	total: 2.18s	remaining: 2m 16s
20:	test: 0.8573547	best: 0.8577007 (18)	total: 4.45s	remaining: 2m 23s
30:	test: 0.8587922	best: 0.8587922 (30)	total: 6.71s	remaining: 2m 24s
40:	test: 0.8592030	best: 0.8593116 (33)	total: 8.99s	remaining: 2m 24s
50:	test: 0.8606605	best: 0.8606605 (50)	total: 11.2s	remaining: 2m 22s
60:	test: 0.8615295	best: 0.8615628 (57)	total: 13.5s	remaining: 2m 21s
70:	test: 0.8623809	best: 0.8624264 (66)	total: 15.7s	remaining: 2m 19s
80:	test: 0.8615890	best: 0.8624913 (71)	total: 18s	remaining: 2m 17s
90:	test: 0.8611914	best: 0.8624913 (71)	total: 20.3s	remaining: 2m 15s
100:	test: 0.8613070	best: 0.8624913 (71)	total: 22.6s	remaining: 2m 13s
110:	test: 0.8614699	best: 0.8624913 (71)	total: 24.8s	remaining: 2m 11s
120:	test: 0.8608707	best: 0.8624913 (71)	total: 27.1s	remaining: 2m 9s
130:	test: 0.8605344	best: 0.8624913 (71)	total: 29.2s	remaining: 

In [53]:
X=X_sample1.copy()
y=y_sample1.copy()
oof_preds = []
oof_aucs = []
test_pool = cb.Pool(test.astype(str), cat_features=X.columns.values)
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
    X_train_pool = cb.Pool(X_train.astype(str), y_train, cat_features=X.columns.values)
    X_test_pool = cb.Pool(X_test.astype(str), y_test, cat_features=X.columns.values)
    
    cat_clf = cb.CatBoostClassifier(**cat_params)
    cat_clf = cat_clf.fit(X=X_train_pool,
                          eval_set=X_test_pool,
                          verbose=10,
                          early_stopping_rounds=200)
    
    test_pred = cat_clf.predict_proba(test_pool)[:, 1]
    
    oof_preds.append(test_pred)
    auc = cat_clf.best_score_['validation']['AUC']
    oof_aucs.append(auc)
    print(f"\n---- Fold {fold}: ROC-AUC Score: {auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    del X_train_pool, X_test_pool
    del cat_clf
    gc.collect()

auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} \xB1 {auc_std:.6f}\n")

test_pred_cat = np.mean(oof_preds, axis=0)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8193126	best: 0.8193126 (0)	total: 30.4ms	remaining: 21.2s
10:	test: 0.8160504	best: 0.8193126 (0)	total: 332ms	remaining: 20.8s
20:	test: 0.8312511	best: 0.8312511 (20)	total: 2.26s	remaining: 1m 13s
30:	test: 0.8374296	best: 0.8374979 (26)	total: 4.45s	remaining: 1m 35s
40:	test: 0.8373359	best: 0.8378839 (35)	total: 6.3s	remaining: 1m 41s
50:	test: 0.8375057	best: 0.8382114 (47)	total: 8.5s	remaining: 1m 48s
60:	test: 0.8373009	best: 0.8382114 (47)	total: 10.3s	remaining: 1m 48s
70:	test: 0.8369244	best: 0.8382114 (47)	total: 12.5s	remaining: 1m 51s
80:	test: 0.8373779	best: 0.8382114 (47)	total: 14.2s	remaining: 1m 48s
90:	test: 0.8371380	best: 0.8382114 (47)	total: 16.1s	remaining: 1m 47s
100:	test: 0.8372011	best: 0.8382114 (47)	total: 17.1s	remaining: 1m 41s
110:	test: 0.8378647	best: 0.8382114 (47)	total: 18.4s	remaining: 1m 37s
120:	test: 0.8378384	best: 0.8382114 (47)	total: 19.9s	remaining: 1m 35s
130:	test: 0.8377894	best: 0.8382114 (47)	total: 21.5s	remaining: 1

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8247548	best: 0.8264235 (8)	total: 506ms	remaining: 31.7s
20:	test: 0.8316407	best: 0.8316407 (20)	total: 2.05s	remaining: 1m 6s
30:	test: 0.8326186	best: 0.8326186 (30)	total: 4.17s	remaining: 1m 29s
40:	test: 0.8322229	best: 0.8326186 (30)	total: 6.38s	remaining: 1m 42s
50:	test: 0.8330284	best: 0.8331807 (49)	total: 8.59s	remaining: 1m 49s
60:	test: 0.8366075	best: 0.8367475 (57)	total: 10.6s	remaining: 1m 51s
70:	test: 0.8377684	best: 0.8380188 (69)	total: 12.7s	remaining: 1m 52s
80:	test: 0.8380258	best: 0.8380258 (80)	total: 14.6s	remaining: 1m 51s
90:	test: 0.8367090	best: 0.8380258 (80)	total: 16s	remaining: 1m 47s
100:	test: 0.8374532	best: 0.8380258 (80)	total: 17.6s	remaining: 1m 44s
110:	test: 0.8360296	best: 0.8380258 (80)	total: 19.4s	remaining: 1m 42s
120:	test: 0.8358055	best: 0.8380258 (80)	total: 21.3s	remaining: 1m 41s
130:	test: 0.8358160	best: 0.8380258 (80)	total: 23.3s	remaining: 1m 41s
140:	test: 0.8354763	best: 0.8380258 (80)	total: 25.3s	remaining:

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8355682	best: 0.8355682 (10)	total: 473ms	remaining: 29.7s
20:	test: 0.8384670	best: 0.8394424 (19)	total: 2.62s	remaining: 1m 24s
30:	test: 0.8439407	best: 0.8445125 (29)	total: 4.82s	remaining: 1m 44s
40:	test: 0.8448968	best: 0.8452085 (37)	total: 7.03s	remaining: 1m 52s
50:	test: 0.8454992	best: 0.8455009 (48)	total: 9.22s	remaining: 1m 57s
60:	test: 0.8454379	best: 0.8455009 (48)	total: 11.4s	remaining: 1m 59s
70:	test: 0.8452698	best: 0.8455009 (48)	total: 13.4s	remaining: 1m 58s
80:	test: 0.8455394	best: 0.8456182 (78)	total: 15.3s	remaining: 1m 56s
90:	test: 0.8454256	best: 0.8456182 (78)	total: 16.7s	remaining: 1m 51s
100:	test: 0.8454589	best: 0.8456182 (78)	total: 18.1s	remaining: 1m 47s
110:	test: 0.8452593	best: 0.8456182 (78)	total: 20.1s	remaining: 1m 46s
120:	test: 0.8450824	best: 0.8456182 (78)	total: 21.9s	remaining: 1m 44s
130:	test: 0.8443225	best: 0.8456182 (78)	total: 24s	remaining: 1m 44s
140:	test: 0.8442542	best: 0.8456182 (78)	total: 24.9s	remainin

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8197897	best: 0.8197897 (10)	total: 595ms	remaining: 37.2s
20:	test: 0.8283155	best: 0.8283155 (20)	total: 2.23s	remaining: 1m 12s
30:	test: 0.8267264	best: 0.8291166 (26)	total: 4.22s	remaining: 1m 31s
40:	test: 0.8265575	best: 0.8291166 (26)	total: 6.26s	remaining: 1m 40s
50:	test: 0.8259428	best: 0.8291166 (26)	total: 8.22s	remaining: 1m 44s
60:	test: 0.8254666	best: 0.8291166 (26)	total: 9.89s	remaining: 1m 43s
70:	test: 0.8262221	best: 0.8291166 (26)	total: 11.9s	remaining: 1m 45s
80:	test: 0.8259122	best: 0.8291166 (26)	total: 14s	remaining: 1m 47s
90:	test: 0.8265671	best: 0.8291166 (26)	total: 15.8s	remaining: 1m 46s
100:	test: 0.8267965	best: 0.8291166 (26)	total: 17.7s	remaining: 1m 44s
110:	test: 0.8273428	best: 0.8291166 (26)	total: 19.9s	remaining: 1m 45s
120:	test: 0.8268017	best: 0.8291166 (26)	total: 21.9s	remaining: 1m 44s
130:	test: 0.8286963	best: 0.8291166 (26)	total: 23.6s	remaining: 1m 42s
140:	test: 0.8290780	best: 0.8292269 (137)	total: 25.2s	remaini

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8198992	best: 0.8198992 (10)	total: 577ms	remaining: 36.1s
20:	test: 0.8270434	best: 0.8270951 (19)	total: 1.62s	remaining: 52.3s
30:	test: 0.8267263	best: 0.8279360 (24)	total: 3.81s	remaining: 1m 22s
40:	test: 0.8276908	best: 0.8284748 (35)	total: 5.99s	remaining: 1m 36s
50:	test: 0.8281647	best: 0.8284748 (35)	total: 7.45s	remaining: 1m 34s
60:	test: 0.8275480	best: 0.8284748 (35)	total: 8.96s	remaining: 1m 33s
70:	test: 0.8296792	best: 0.8296792 (70)	total: 11s	remaining: 1m 37s
80:	test: 0.8305762	best: 0.8306848 (79)	total: 12.6s	remaining: 1m 36s
90:	test: 0.8303186	best: 0.8306848 (79)	total: 14.8s	remaining: 1m 39s
100:	test: 0.8305236	best: 0.8306848 (79)	total: 16.9s	remaining: 1m 40s
110:	test: 0.8306042	best: 0.8306848 (79)	total: 18.9s	remaining: 1m 40s
120:	test: 0.8305639	best: 0.8306848 (79)	total: 21s	remaining: 1m 40s
130:	test: 0.8303887	best: 0.8306848 (79)	total: 22.4s	remaining: 1m 37s
140:	test: 0.8302170	best: 0.8306848 (79)	total: 23.9s	remaining: 

In [54]:
X=X_sample2.copy()
y=y_sample2.copy()
oof_preds = []
oof_aucs = []
test_pool = cb.Pool(test.astype(str), cat_features=X.columns.values)
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
    X_train_pool = cb.Pool(X_train.astype(str), y_train, cat_features=X.columns.values)
    X_test_pool = cb.Pool(X_test.astype(str), y_test, cat_features=X.columns.values)
    
    cat_clf = cb.CatBoostClassifier(**cat_params)
    cat_clf = cat_clf.fit(X=X_train_pool,
                          eval_set=X_test_pool,
                          verbose=10,
                          early_stopping_rounds=200)
    
    test_pred = cat_clf.predict_proba(test_pool)[:, 1]
    
    oof_preds.append(test_pred)
    auc = cat_clf.best_score_['validation']['AUC']
    oof_aucs.append(auc)
    print(f"\n---- Fold {fold}: ROC-AUC Score: {auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    del X_train_pool, X_test_pool
    del cat_clf
    gc.collect()

auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} \xB1 {auc_std:.6f}\n")

test_pred_cat = np.mean(oof_preds, axis=0)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8193126	best: 0.8193126 (0)	total: 31.8ms	remaining: 22.3s
10:	test: 0.8441553	best: 0.8442078 (9)	total: 1.7s	remaining: 1m 46s
20:	test: 0.8510079	best: 0.8510079 (20)	total: 3.95s	remaining: 2m 7s
30:	test: 0.8517687	best: 0.8517687 (30)	total: 6.17s	remaining: 2m 13s
40:	test: 0.8518633	best: 0.8519097 (31)	total: 8.39s	remaining: 2m 14s
50:	test: 0.8537036	best: 0.8537036 (50)	total: 10.6s	remaining: 2m 14s
60:	test: 0.8543515	best: 0.8545073 (56)	total: 12.8s	remaining: 2m 13s
70:	test: 0.8529401	best: 0.8545073 (56)	total: 15s	remaining: 2m 12s
80:	test: 0.8523115	best: 0.8545073 (56)	total: 17.2s	remaining: 2m 11s
90:	test: 0.8520366	best: 0.8545073 (56)	total: 19.2s	remaining: 2m 8s
100:	test: 0.8518423	best: 0.8545073 (56)	total: 21.3s	remaining: 2m 6s
110:	test: 0.8519981	best: 0.8545073 (56)	total: 23.3s	remaining: 2m 3s
120:	test: 0.8524464	best: 0.8545073 (56)	total: 25.3s	remaining: 2m 1s
130:	test: 0.8523921	best: 0.8545073 (56)	total: 27.4s	remaining: 1m 59s

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8474629	best: 0.8480933 (9)	total: 1.14s	remaining: 1m 11s
20:	test: 0.8529489	best: 0.8529988 (15)	total: 3.25s	remaining: 1m 45s
30:	test: 0.8541063	best: 0.8541063 (30)	total: 5.48s	remaining: 1m 58s
40:	test: 0.8515831	best: 0.8550729 (32)	total: 7.53s	remaining: 2m
50:	test: 0.8527528	best: 0.8550729 (32)	total: 9.72s	remaining: 2m 3s
60:	test: 0.8534147	best: 0.8550729 (32)	total: 11.9s	remaining: 2m 4s
70:	test: 0.8540389	best: 0.8550729 (32)	total: 14s	remaining: 2m 4s
80:	test: 0.8540205	best: 0.8550729 (32)	total: 16.2s	remaining: 2m 4s
90:	test: 0.8543287	best: 0.8550729 (32)	total: 18.4s	remaining: 2m 3s
100:	test: 0.8540258	best: 0.8550729 (32)	total: 20.4s	remaining: 2m 1s
110:	test: 0.8537579	best: 0.8550729 (32)	total: 22.5s	remaining: 1m 59s
120:	test: 0.8541878	best: 0.8550729 (32)	total: 24.3s	remaining: 1m 56s
130:	test: 0.8544898	best: 0.8550729 (32)	total: 26.5s	remaining: 1m 54s
140:	test: 0.8546439	best: 0.8550729 (32)	total: 28.6s	remaining: 1m 53s


Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8487298	best: 0.8487298 (10)	total: 1.69s	remaining: 1m 45s
20:	test: 0.8542473	best: 0.8553566 (14)	total: 3.92s	remaining: 2m 6s
30:	test: 0.8572591	best: 0.8574560 (28)	total: 6.15s	remaining: 2m 12s
40:	test: 0.8568108	best: 0.8576907 (35)	total: 8.34s	remaining: 2m 14s
50:	test: 0.8565595	best: 0.8576907 (35)	total: 10.5s	remaining: 2m 14s
60:	test: 0.8568449	best: 0.8576907 (35)	total: 12.6s	remaining: 2m 11s
70:	test: 0.8552419	best: 0.8576907 (35)	total: 14.6s	remaining: 2m 9s
80:	test: 0.8547621	best: 0.8576907 (35)	total: 16.6s	remaining: 2m 6s
90:	test: 0.8544837	best: 0.8576907 (35)	total: 18.6s	remaining: 2m 4s
100:	test: 0.8540879	best: 0.8576907 (35)	total: 20.6s	remaining: 2m 2s
110:	test: 0.8546605	best: 0.8576907 (35)	total: 22.7s	remaining: 2m
120:	test: 0.8544802	best: 0.8576907 (35)	total: 24.2s	remaining: 1m 55s
130:	test: 0.8546535	best: 0.8576907 (35)	total: 26.1s	remaining: 1m 53s
140:	test: 0.8547078	best: 0.8576907 (35)	total: 27.8s	remaining: 1m 

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8425995	best: 0.8427824 (8)	total: 1.34s	remaining: 1m 23s
20:	test: 0.8457706	best: 0.8458284 (19)	total: 3.57s	remaining: 1m 55s
30:	test: 0.8455780	best: 0.8461759 (21)	total: 5.66s	remaining: 2m 2s
40:	test: 0.8445869	best: 0.8461759 (21)	total: 7.87s	remaining: 2m 6s
50:	test: 0.8452199	best: 0.8461759 (21)	total: 9.98s	remaining: 2m 7s
60:	test: 0.8450115	best: 0.8461759 (21)	total: 11.9s	remaining: 2m 5s
70:	test: 0.8435836	best: 0.8461759 (21)	total: 14.1s	remaining: 2m 4s
80:	test: 0.8445291	best: 0.8461759 (21)	total: 16.2s	remaining: 2m 3s
90:	test: 0.8446254	best: 0.8461759 (21)	total: 18.1s	remaining: 2m 1s
100:	test: 0.8436212	best: 0.8461759 (21)	total: 19.8s	remaining: 1m 57s
110:	test: 0.8432482	best: 0.8461759 (21)	total: 21.7s	remaining: 1m 55s
120:	test: 0.8435459	best: 0.8461759 (21)	total: 23.9s	remaining: 1m 54s
130:	test: 0.8439661	best: 0.8461759 (21)	total: 26.1s	remaining: 1m 53s
140:	test: 0.8437403	best: 0.8461759 (21)	total: 28.2s	remaining: 1m

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8339179	best: 0.8339179 (10)	total: 1.13s	remaining: 1m 11s
20:	test: 0.8449058	best: 0.8452176 (19)	total: 3.35s	remaining: 1m 48s
30:	test: 0.8469134	best: 0.8469134 (30)	total: 5.57s	remaining: 2m
40:	test: 0.8476921	best: 0.8481038 (37)	total: 7.77s	remaining: 2m 4s
50:	test: 0.8486049	best: 0.8486049 (50)	total: 9.99s	remaining: 2m 7s
60:	test: 0.8501150	best: 0.8501754 (55)	total: 11.7s	remaining: 2m 2s
70:	test: 0.8501439	best: 0.8501781 (61)	total: 13.8s	remaining: 2m 2s
80:	test: 0.8502613	best: 0.8502613 (80)	total: 15.9s	remaining: 2m 1s
90:	test: 0.8503962	best: 0.8505556 (86)	total: 17.9s	remaining: 1m 59s
100:	test: 0.8496691	best: 0.8505556 (86)	total: 20s	remaining: 1m 58s
110:	test: 0.8495290	best: 0.8505556 (86)	total: 22.2s	remaining: 1m 57s
120:	test: 0.8498444	best: 0.8505556 (86)	total: 24.4s	remaining: 1m 56s
130:	test: 0.8499284	best: 0.8505556 (86)	total: 26.4s	remaining: 1m 54s
140:	test: 0.8496779	best: 0.8505556 (86)	total: 28.6s	remaining: 1m 53

In [55]:
X=X_sample3.copy()
y=y_sample3.copy()
oof_preds = []
oof_aucs = []
test_pool = cb.Pool(test.astype(str), cat_features=X.columns.values)
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
    X_train_pool = cb.Pool(X_train.astype(str), y_train, cat_features=X.columns.values)
    X_test_pool = cb.Pool(X_test.astype(str), y_test, cat_features=X.columns.values)
    
    cat_clf = cb.CatBoostClassifier(**cat_params)
    cat_clf = cat_clf.fit(X=X_train_pool,
                          eval_set=X_test_pool,
                          verbose=10,
                          early_stopping_rounds=200)
    
    test_pred = cat_clf.predict_proba(test_pool)[:, 1]
    
    oof_preds.append(test_pred)
    auc = cat_clf.best_score_['validation']['AUC']
    oof_aucs.append(auc)
    print(f"\n---- Fold {fold}: ROC-AUC Score: {auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    del X_train_pool, X_test_pool
    del cat_clf
    gc.collect()

auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} \xB1 {auc_std:.6f}\n")

test_pred_cat = np.mean(oof_preds, axis=0)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8193126	best: 0.8193126 (0)	total: 31.4ms	remaining: 21.9s
10:	test: 0.8601141	best: 0.8601141 (10)	total: 1.72s	remaining: 1m 47s
20:	test: 0.8627433	best: 0.8627433 (20)	total: 3.99s	remaining: 2m 9s
30:	test: 0.8642150	best: 0.8642150 (30)	total: 6.23s	remaining: 2m 14s
40:	test: 0.8644628	best: 0.8646204 (39)	total: 8.44s	remaining: 2m 15s
50:	test: 0.8649075	best: 0.8650038 (49)	total: 10.7s	remaining: 2m 15s
60:	test: 0.8635619	best: 0.8650038 (49)	total: 12.9s	remaining: 2m 15s
70:	test: 0.8635058	best: 0.8650038 (49)	total: 15.1s	remaining: 2m 14s
80:	test: 0.8632309	best: 0.8650038 (49)	total: 17.4s	remaining: 2m 12s
90:	test: 0.8635181	best: 0.8650038 (49)	total: 19.6s	remaining: 2m 11s
100:	test: 0.8635479	best: 0.8650038 (49)	total: 21.3s	remaining: 2m 6s
110:	test: 0.8635916	best: 0.8650038 (49)	total: 23.2s	remaining: 2m 3s
120:	test: 0.8636162	best: 0.8650038 (49)	total: 24.7s	remaining: 1m 58s
130:	test: 0.8635461	best: 0.8650038 (49)	total: 26.7s	remaining: 

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8537290	best: 0.8537290 (10)	total: 1.56s	remaining: 1m 37s
20:	test: 0.8600572	best: 0.8600572 (20)	total: 3.54s	remaining: 1m 54s
30:	test: 0.8611533	best: 0.8615071 (22)	total: 5.75s	remaining: 2m 4s
40:	test: 0.8604888	best: 0.8615071 (22)	total: 7.97s	remaining: 2m 8s
50:	test: 0.8611148	best: 0.8615071 (22)	total: 10.2s	remaining: 2m 9s
60:	test: 0.8598567	best: 0.8615071 (22)	total: 12.3s	remaining: 2m 8s
70:	test: 0.8598558	best: 0.8615071 (22)	total: 14.3s	remaining: 2m 6s
80:	test: 0.8595967	best: 0.8615071 (22)	total: 16.5s	remaining: 2m 6s
90:	test: 0.8601027	best: 0.8615071 (22)	total: 18.8s	remaining: 2m 5s
100:	test: 0.8607164	best: 0.8615071 (22)	total: 21s	remaining: 2m 4s
110:	test: 0.8603102	best: 0.8615071 (22)	total: 23.2s	remaining: 2m 3s
120:	test: 0.8602227	best: 0.8615071 (22)	total: 25.3s	remaining: 2m
130:	test: 0.8609992	best: 0.8615071 (22)	total: 27.4s	remaining: 1m 59s
140:	test: 0.8604459	best: 0.8615071 (22)	total: 29.7s	remaining: 1m 57s
15

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8659975	best: 0.8659975 (10)	total: 1.67s	remaining: 1m 44s
20:	test: 0.8719580	best: 0.8719580 (20)	total: 3.91s	remaining: 2m 6s
30:	test: 0.8737126	best: 0.8737353 (26)	total: 6.11s	remaining: 2m 11s
40:	test: 0.8718871	best: 0.8737353 (26)	total: 8.36s	remaining: 2m 14s
50:	test: 0.8722863	best: 0.8737353 (26)	total: 10.6s	remaining: 2m 14s
60:	test: 0.8726515	best: 0.8737353 (26)	total: 12.8s	remaining: 2m 14s
70:	test: 0.8729491	best: 0.8737353 (26)	total: 14.6s	remaining: 2m 9s
80:	test: 0.8728948	best: 0.8737353 (26)	total: 16.8s	remaining: 2m 8s
90:	test: 0.8727565	best: 0.8737353 (26)	total: 18.8s	remaining: 2m 5s
100:	test: 0.8724483	best: 0.8737353 (26)	total: 20.5s	remaining: 2m 1s
110:	test: 0.8722872	best: 0.8737353 (26)	total: 22.5s	remaining: 1m 59s
120:	test: 0.8720141	best: 0.8737353 (26)	total: 24.7s	remaining: 1m 58s
130:	test: 0.8719318	best: 0.8737353 (26)	total: 26.7s	remaining: 1m 55s
140:	test: 0.8719650	best: 0.8737353 (26)	total: 28.5s	remaining:

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8597945	best: 0.8597945 (10)	total: 1.74s	remaining: 1m 49s
20:	test: 0.8643008	best: 0.8643008 (20)	total: 3.98s	remaining: 2m 8s
30:	test: 0.8645459	best: 0.8652061 (25)	total: 6.2s	remaining: 2m 13s
40:	test: 0.8637790	best: 0.8652061 (25)	total: 8.41s	remaining: 2m 15s
50:	test: 0.8635697	best: 0.8652061 (25)	total: 10.6s	remaining: 2m 15s
60:	test: 0.8629499	best: 0.8652061 (25)	total: 12.9s	remaining: 2m 14s
70:	test: 0.8612277	best: 0.8652061 (25)	total: 15.1s	remaining: 2m 13s
80:	test: 0.8615395	best: 0.8652061 (25)	total: 17.3s	remaining: 2m 12s
90:	test: 0.8611516	best: 0.8652061 (25)	total: 19.5s	remaining: 2m 10s
100:	test: 0.8606280	best: 0.8652061 (25)	total: 21.4s	remaining: 2m 7s
110:	test: 0.8600554	best: 0.8652061 (25)	total: 23.6s	remaining: 2m 5s
120:	test: 0.8601010	best: 0.8652061 (25)	total: 25.7s	remaining: 2m 3s
130:	test: 0.8600607	best: 0.8652061 (25)	total: 27.8s	remaining: 2m
140:	test: 0.8598094	best: 0.8652061 (25)	total: 30.1s	remaining: 1m 

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8524126	best: 0.8524126 (10)	total: 1.46s	remaining: 1m 31s
20:	test: 0.8569938	best: 0.8571962 (18)	total: 3.71s	remaining: 2m
30:	test: 0.8574546	best: 0.8580958 (22)	total: 5.95s	remaining: 2m 8s
40:	test: 0.8573862	best: 0.8584076 (37)	total: 7.89s	remaining: 2m 6s
50:	test: 0.8579153	best: 0.8584076 (37)	total: 10.1s	remaining: 2m 8s
60:	test: 0.8584847	best: 0.8584847 (59)	total: 12.3s	remaining: 2m 9s
70:	test: 0.8583261	best: 0.8584882 (61)	total: 14.6s	remaining: 2m 8s
80:	test: 0.8583261	best: 0.8586047 (72)	total: 16.6s	remaining: 2m 6s
90:	test: 0.8586678	best: 0.8586678 (90)	total: 18.8s	remaining: 2m 5s
100:	test: 0.8587413	best: 0.8588149 (96)	total: 21s	remaining: 2m 4s
110:	test: 0.8589743	best: 0.8592967 (109)	total: 23.2s	remaining: 2m 3s
120:	test: 0.8584094	best: 0.8592967 (109)	total: 25.4s	remaining: 2m 1s
130:	test: 0.8573354	best: 0.8592967 (109)	total: 27.6s	remaining: 2m
140:	test: 0.8570499	best: 0.8592967 (109)	total: 29.9s	remaining: 1m 58s
150

In [56]:
X=X_sample4.copy()
y=y_sample4.copy()
oof_preds = []
oof_aucs = []
test_pool = cb.Pool(test.astype(str), cat_features=X.columns.values)
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
    X_train_pool = cb.Pool(X_train.astype(str), y_train, cat_features=X.columns.values)
    X_test_pool = cb.Pool(X_test.astype(str), y_test, cat_features=X.columns.values)
    
    cat_clf = cb.CatBoostClassifier(**cat_params)
    cat_clf = cat_clf.fit(X=X_train_pool,
                          eval_set=X_test_pool,
                          verbose=10,
                          early_stopping_rounds=200)
    
    test_pred = cat_clf.predict_proba(test_pool)[:, 1]
    
    oof_preds.append(test_pred)
    auc = cat_clf.best_score_['validation']['AUC']
    oof_aucs.append(auc)
    print(f"\n---- Fold {fold}: ROC-AUC Score: {auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    del X_train_pool, X_test_pool
    del cat_clf
    gc.collect()

auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} \xB1 {auc_std:.6f}\n")

test_pred_cat = np.mean(oof_preds, axis=0)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8327298	best: 0.8327298 (0)	total: 30.8ms	remaining: 21.5s
10:	test: 0.8619606	best: 0.8619606 (10)	total: 1.77s	remaining: 1m 50s
20:	test: 0.8619728	best: 0.8631442 (13)	total: 4s	remaining: 2m 9s
30:	test: 0.8633815	best: 0.8642264 (24)	total: 6.26s	remaining: 2m 15s
40:	test: 0.8631977	best: 0.8642264 (24)	total: 8.47s	remaining: 2m 16s
50:	test: 0.8636914	best: 0.8642264 (24)	total: 10.7s	remaining: 2m 16s
60:	test: 0.8635750	best: 0.8642264 (24)	total: 12.9s	remaining: 2m 15s
70:	test: 0.8635006	best: 0.8642264 (24)	total: 15.2s	remaining: 2m 14s
80:	test: 0.8636074	best: 0.8642264 (24)	total: 17.4s	remaining: 2m 13s
90:	test: 0.8640846	best: 0.8642299 (87)	total: 19.5s	remaining: 2m 10s
100:	test: 0.8644943	best: 0.8645363 (99)	total: 21.5s	remaining: 2m 7s
110:	test: 0.8647780	best: 0.8650581 (108)	total: 23.7s	remaining: 2m 5s
120:	test: 0.8643122	best: 0.8650581 (108)	total: 25.9s	remaining: 2m 4s
130:	test: 0.8643875	best: 0.8650581 (108)	total: 27.9s	remaining: 2

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8368132	best: 0.8368132 (0)	total: 56.4ms	remaining: 39.4s
10:	test: 0.8544530	best: 0.8544530 (10)	total: 1.63s	remaining: 1m 42s
20:	test: 0.8586537	best: 0.8586537 (20)	total: 3.86s	remaining: 2m 4s
30:	test: 0.8608399	best: 0.8608399 (30)	total: 6.09s	remaining: 2m 11s
40:	test: 0.8630007	best: 0.8630007 (40)	total: 8.31s	remaining: 2m 13s
50:	test: 0.8632353	best: 0.8632353 (50)	total: 10.5s	remaining: 2m 14s
60:	test: 0.8625944	best: 0.8633544 (55)	total: 12.6s	remaining: 2m 11s
70:	test: 0.8622285	best: 0.8633544 (55)	total: 14.8s	remaining: 2m 11s
80:	test: 0.8612041	best: 0.8633544 (55)	total: 17s	remaining: 2m 10s
90:	test: 0.8611787	best: 0.8633544 (55)	total: 19.3s	remaining: 2m 9s
100:	test: 0.8598830	best: 0.8633544 (55)	total: 21.4s	remaining: 2m 6s
110:	test: 0.8592456	best: 0.8633544 (55)	total: 23.6s	remaining: 2m 5s
120:	test: 0.8593069	best: 0.8633544 (55)	total: 25.8s	remaining: 2m 3s
130:	test: 0.8603452	best: 0.8633544 (55)	total: 28s	remaining: 2m 1s


Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8668398	best: 0.8668398 (10)	total: 1.84s	remaining: 1m 55s
20:	test: 0.8719607	best: 0.8721051 (17)	total: 4.13s	remaining: 2m 13s
30:	test: 0.8730235	best: 0.8739708 (25)	total: 6.36s	remaining: 2m 17s
40:	test: 0.8731505	best: 0.8739708 (25)	total: 8.6s	remaining: 2m 18s
50:	test: 0.8718381	best: 0.8739708 (25)	total: 10.8s	remaining: 2m 17s
60:	test: 0.8725359	best: 0.8739708 (25)	total: 13s	remaining: 2m 16s
70:	test: 0.8716980	best: 0.8739708 (25)	total: 15.2s	remaining: 2m 14s
80:	test: 0.8719414	best: 0.8739708 (25)	total: 17.4s	remaining: 2m 12s
90:	test: 0.8721813	best: 0.8739708 (25)	total: 19.5s	remaining: 2m 10s
100:	test: 0.8721970	best: 0.8739708 (25)	total: 21.4s	remaining: 2m 6s
110:	test: 0.8726541	best: 0.8739708 (25)	total: 23.6s	remaining: 2m 5s
120:	test: 0.8727609	best: 0.8739708 (25)	total: 25.8s	remaining: 2m 3s
130:	test: 0.8726085	best: 0.8739708 (25)	total: 27.9s	remaining: 2m 1s
140:	test: 0.8723231	best: 0.8739708 (25)	total: 30s	remaining: 1m 

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8612321	best: 0.8612321 (10)	total: 1.44s	remaining: 1m 30s
20:	test: 0.8638919	best: 0.8638919 (20)	total: 3.68s	remaining: 1m 58s
30:	test: 0.8653155	best: 0.8654626 (28)	total: 5.92s	remaining: 2m 7s
40:	test: 0.8656386	best: 0.8658697 (35)	total: 8.15s	remaining: 2m 11s
50:	test: 0.8649399	best: 0.8658697 (35)	total: 10.4s	remaining: 2m 12s
60:	test: 0.8650275	best: 0.8658697 (35)	total: 12.6s	remaining: 2m 12s
70:	test: 0.8651746	best: 0.8658697 (35)	total: 14.9s	remaining: 2m 12s
80:	test: 0.8642596	best: 0.8658697 (35)	total: 17.2s	remaining: 2m 11s
90:	test: 0.8645162	best: 0.8658697 (35)	total: 19.2s	remaining: 2m 8s
100:	test: 0.8643849	best: 0.8658697 (35)	total: 21.3s	remaining: 2m 6s
110:	test: 0.8646002	best: 0.8658697 (35)	total: 23.5s	remaining: 2m 4s
120:	test: 0.8649364	best: 0.8658697 (35)	total: 25.8s	remaining: 2m 3s
130:	test: 0.8655458	best: 0.8658697 (35)	total: 28s	remaining: 2m 1s
140:	test: 0.8644181	best: 0.8658697 (35)	total: 30.1s	remaining: 1m

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8542591	best: 0.8564718 (6)	total: 1.77s	remaining: 1m 50s
20:	test: 0.8571962	best: 0.8579162 (19)	total: 4s	remaining: 2m 9s
30:	test: 0.8571874	best: 0.8579162 (19)	total: 6.22s	remaining: 2m 14s
40:	test: 0.8572829	best: 0.8579162 (19)	total: 8.46s	remaining: 2m 15s
50:	test: 0.8584278	best: 0.8584278 (50)	total: 10.7s	remaining: 2m 15s
60:	test: 0.8587475	best: 0.8587475 (60)	total: 12.9s	remaining: 2m 15s
70:	test: 0.8594500	best: 0.8595709 (68)	total: 15.1s	remaining: 2m 14s
80:	test: 0.8594727	best: 0.8596751 (72)	total: 17.4s	remaining: 2m 12s
90:	test: 0.8594780	best: 0.8596751 (72)	total: 19.6s	remaining: 2m 11s
100:	test: 0.8598599	best: 0.8598599 (99)	total: 21.7s	remaining: 2m 8s
110:	test: 0.8603242	best: 0.8603487 (107)	total: 24s	remaining: 2m 7s
120:	test: 0.8603110	best: 0.8603811 (119)	total: 26.2s	remaining: 2m 5s
130:	test: 0.8607035	best: 0.8607052 (129)	total: 28.4s	remaining: 2m 3s
140:	test: 0.8612641	best: 0.8612868 (138)	total: 30.7s	remaining: 2

In [57]:
X=X_sample5.copy()
y=y_sample5.copy()
oof_preds = []
oof_aucs = []
test_pool = cb.Pool(test.astype(str), cat_features=X.columns.values)
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
    
    X_train_pool = cb.Pool(X_train.astype(str), y_train, cat_features=X.columns.values)
    X_test_pool = cb.Pool(X_test.astype(str), y_test, cat_features=X.columns.values)
    
    cat_clf = cb.CatBoostClassifier(**cat_params)
    cat_clf = cat_clf.fit(X=X_train_pool,
                          eval_set=X_test_pool,
                          verbose=10,
                          early_stopping_rounds=200)
    
    test_pred = cat_clf.predict_proba(test_pool)[:, 1]
    
    oof_preds.append(test_pred)
    auc = cat_clf.best_score_['validation']['AUC']
    oof_aucs.append(auc)
    print(f"\n---- Fold {fold}: ROC-AUC Score: {auc:.6f}\n")
    
    del X_train, y_train, X_test, y_test
    del X_train_pool, X_test_pool
    del cat_clf
    gc.collect()

auc_mean = np.mean(oof_aucs)
auc_std = np.std(oof_aucs)
print(f"\n---> ROC-AUC Score: {auc_mean:.6f} \xB1 {auc_std:.6f}\n")

test_pred_cat = np.mean(oof_preds, axis=0)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8327298	best: 0.8327298 (0)	total: 34.5ms	remaining: 24.1s
10:	test: 0.8610159	best: 0.8610159 (10)	total: 1.84s	remaining: 1m 55s
20:	test: 0.8626793	best: 0.8626793 (20)	total: 4.11s	remaining: 2m 12s
30:	test: 0.8632388	best: 0.8632633 (25)	total: 6.33s	remaining: 2m 16s
40:	test: 0.8627450	best: 0.8632633 (25)	total: 8.56s	remaining: 2m 17s
50:	test: 0.8634008	best: 0.8636538 (43)	total: 10.8s	remaining: 2m 17s
60:	test: 0.8641563	best: 0.8641563 (60)	total: 13.1s	remaining: 2m 16s
70:	test: 0.8635750	best: 0.8642772 (62)	total: 15.3s	remaining: 2m 15s
80:	test: 0.8647972	best: 0.8649233 (78)	total: 17.6s	remaining: 2m 14s
90:	test: 0.8651509	best: 0.8652998 (85)	total: 19.8s	remaining: 2m 12s
100:	test: 0.8647604	best: 0.8652998 (85)	total: 22s	remaining: 2m 10s
110:	test: 0.8634104	best: 0.8652998 (85)	total: 24.2s	remaining: 2m 8s
120:	test: 0.8629884	best: 0.8652998 (85)	total: 26.4s	remaining: 2m 6s
130:	test: 0.8629359	best: 0.8652998 (85)	total: 28.6s	remaining: 2

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8368132	best: 0.8368132 (0)	total: 58.2ms	remaining: 40.7s
10:	test: 0.8573597	best: 0.8573597 (10)	total: 1.59s	remaining: 1m 39s
20:	test: 0.8630059	best: 0.8631845 (19)	total: 3.87s	remaining: 2m 5s
30:	test: 0.8649802	best: 0.8650993 (29)	total: 6.11s	remaining: 2m 11s
40:	test: 0.8640128	best: 0.8650993 (29)	total: 8.36s	remaining: 2m 14s
50:	test: 0.8678991	best: 0.8678991 (50)	total: 10.6s	remaining: 2m 14s
60:	test: 0.8677993	best: 0.8682739 (55)	total: 12.9s	remaining: 2m 14s
70:	test: 0.8675936	best: 0.8682739 (55)	total: 15.1s	remaining: 2m 13s
80:	test: 0.8680243	best: 0.8682739 (55)	total: 17.3s	remaining: 2m 12s
90:	test: 0.8669807	best: 0.8682739 (55)	total: 19.6s	remaining: 2m 10s
100:	test: 0.8661788	best: 0.8682739 (55)	total: 21.8s	remaining: 2m 9s
110:	test: 0.8662891	best: 0.8682739 (55)	total: 24.1s	remaining: 2m 7s
120:	test: 0.8666288	best: 0.8682739 (55)	total: 26.2s	remaining: 2m 5s
130:	test: 0.8677722	best: 0.8682739 (55)	total: 28.3s	remaining: 2

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8486300	best: 0.8486300 (0)	total: 40.4ms	remaining: 28.2s
10:	test: 0.8662891	best: 0.8662891 (10)	total: 1.78s	remaining: 1m 51s
20:	test: 0.8746135	best: 0.8746135 (20)	total: 4.05s	remaining: 2m 10s
30:	test: 0.8759189	best: 0.8763724 (29)	total: 6.28s	remaining: 2m 15s
40:	test: 0.8760213	best: 0.8763724 (29)	total: 8.51s	remaining: 2m 16s
50:	test: 0.8755809	best: 0.8763724 (29)	total: 10.8s	remaining: 2m 16s
60:	test: 0.8763522	best: 0.8763724 (29)	total: 13s	remaining: 2m 16s
70:	test: 0.8757989	best: 0.8763724 (29)	total: 15.3s	remaining: 2m 15s
80:	test: 0.8749812	best: 0.8763724 (29)	total: 17.5s	remaining: 2m 13s
90:	test: 0.8753909	best: 0.8763724 (29)	total: 19.7s	remaining: 2m 12s
100:	test: 0.8755914	best: 0.8763724 (29)	total: 21.9s	remaining: 2m 10s
110:	test: 0.8755826	best: 0.8763724 (29)	total: 24.2s	remaining: 2m 8s
120:	test: 0.8753042	best: 0.8763724 (29)	total: 26.5s	remaining: 2m 6s
130:	test: 0.8749348	best: 0.8763724 (29)	total: 28.5s	remaining: 2

Default metric period is 5 because AUC is/are not implemented for GPU


10:	test: 0.8582239	best: 0.8604564 (7)	total: 1.51s	remaining: 1m 34s
20:	test: 0.8619466	best: 0.8620516 (18)	total: 3.76s	remaining: 2m 1s
30:	test: 0.8631285	best: 0.8631285 (30)	total: 5.99s	remaining: 2m 9s
40:	test: 0.8618310	best: 0.8637466 (34)	total: 8.24s	remaining: 2m 12s
50:	test: 0.8631162	best: 0.8637466 (34)	total: 10.5s	remaining: 2m 13s
60:	test: 0.8643402	best: 0.8643402 (60)	total: 12.8s	remaining: 2m 13s
70:	test: 0.8642929	best: 0.8644522 (65)	total: 15s	remaining: 2m 13s
80:	test: 0.8643787	best: 0.8644522 (65)	total: 17.3s	remaining: 2m 11s
90:	test: 0.8644680	best: 0.8647044 (89)	total: 19.5s	remaining: 2m 10s
100:	test: 0.8635417	best: 0.8647044 (89)	total: 21.8s	remaining: 2m 9s
110:	test: 0.8627100	best: 0.8647044 (89)	total: 24.1s	remaining: 2m 7s
120:	test: 0.8627135	best: 0.8647044 (89)	total: 26.3s	remaining: 2m 5s
130:	test: 0.8634262	best: 0.8647044 (89)	total: 28.6s	remaining: 2m 4s
140:	test: 0.8634682	best: 0.8647044 (89)	total: 30.7s	remaining: 2m 

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8129083	best: 0.8129083 (0)	total: 32.6ms	remaining: 22.8s
10:	test: 0.8559120	best: 0.8564718 (6)	total: 1.88s	remaining: 1m 57s
20:	test: 0.8578409	best: 0.8590252 (17)	total: 4.13s	remaining: 2m 13s
30:	test: 0.8589980	best: 0.8593676 (28)	total: 6.39s	remaining: 2m 17s
40:	test: 0.8591951	best: 0.8593676 (28)	total: 8.66s	remaining: 2m 19s
50:	test: 0.8599580	best: 0.8599580 (50)	total: 10.9s	remaining: 2m 18s
60:	test: 0.8599090	best: 0.8600841 (58)	total: 13.1s	remaining: 2m 17s
70:	test: 0.8601726	best: 0.8603268 (65)	total: 15.4s	remaining: 2m 16s
80:	test: 0.8598809	best: 0.8603268 (65)	total: 17.6s	remaining: 2m 14s
90:	test: 0.8598214	best: 0.8603268 (65)	total: 19.8s	remaining: 2m 12s
100:	test: 0.8591399	best: 0.8603268 (65)	total: 22.1s	remaining: 2m 10s
110:	test: 0.8588035	best: 0.8603268 (65)	total: 24.3s	remaining: 2m 9s
120:	test: 0.8594535	best: 0.8603268 (65)	total: 26.4s	remaining: 2m 6s
130:	test: 0.8597653	best: 0.8603268 (65)	total: 28.7s	remaining: 